In [13]:
from pathlib import Path
import json
import socket
from utils import load_image, annotate

if socket.gethostname() == "hctlrds":
    dataset_dir = Path("/mnt/ssd2/xin/data/Liebherr_Product")
    repo_dir = Path("/mnt/ssd2/xin/repo/alp/diversification/dreambooth")
elif socket.gethostname() == "BALI":
    dataset_dir = Path("/home/xin/Datasets/Liebherr_Product")
    repo_dir = Path("/home/xin/repo/alp/diversification/dreambooth")
else:
    raise ValueError("Unknown hostname")

label_dir = dataset_dir / "labels_gen"
image_dir = repo_dir / "generated_data"
image_annotated_dir = repo_dir / "generated_data_annotated"

with open(label_dir / "labels_nms.json") as f:
    labels_nms = json.load(f)

In [15]:
### plot iamges based on labels

for id, ann in labels_nms.items():
    file = id + ".jpg"
    classes = labels_nms[id]["classes"]
    if len(classes) == 0:
        continue
    obj = classes[0]
    img_path = image_dir / obj / file
    annotated_img_path = image_annotated_dir / obj / file
    # resume
    if annotated_img_path.exists():
        continue
    if not img_path.exists():
        print(f"{img_path} does not exist")
        continue
    boxes_xywh = labels_nms[id]["boxes"]
    if len(boxes_xywh) == 0:
        continue

    logits = labels_nms[id]["logits"]
    classes = labels_nms[id]["classes"]
    img = load_image(img_path, size=512)
    annotated_frame = annotate(img, boxes_xywh, logits, classes)
    annotated_img_path.parent.mkdir(exist_ok=True, parents=True)
    annotated_frame.save(annotated_img_path)